<a href="https://colab.research.google.com/github/jasperSha/cloud_music/blob/main/cloudmusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [4]:
!gcloud projects list

PROJECT_ID             NAME                         PROJECT_NUMBER
cloudmusic-330004      cloudmusic                   731970834007
cluster-music          cluster-music                850659529381
composed-field-268617  ReverseAddressLookUp         1042903851702
solar-theory-326420    natural-language-processing  128859147416
studenthelper-520f1    studenthelper                280871841346


In [5]:
!gcloud config set project cluster-music

Updated property [core/project].


In [ ]:
!gsutil ls gs://deepclustermusic

gs://deepclustermusic/ArQAeJKUhc0.png


In [1]:
import os
# create virtual directory for image data
os.makedirs('song_images', exist_ok=True)

# create virtual directory for frequency data
os.makedirs('song_freqs', exist_ok=True)

# create virtual directory for the flac files
os.makedirs('song_flacs', exist_ok=True)

In [6]:
# load images
!gsutil -m cp gs://deepclustermusic/flac_files/*.flac './song_flacs'

# load frequency data
# !gsutil cp gs://deepclustermusic/*.csv './song_freqs'

Copying gs://deepclustermusic/flac_files/-08w0k42lx4.flac...
Copying gs://deepclustermusic/flac_files/-0AGRrdWs9c.flac...
Copying gs://deepclustermusic/flac_files/-0PY5uFqQK0.flac...
Copying gs://deepclustermusic/flac_files/-0rEs3zid2U.flac...
Copying gs://deepclustermusic/flac_files/0-9-hbvCHtE.flac...
Copying gs://deepclustermusic/flac_files/00F3w0y38nI.flac...
Copying gs://deepclustermusic/flac_files/00Rg2JMkFY0.flac...
Copying gs://deepclustermusic/flac_files/00b--K1VQao.flac...
Copying gs://deepclustermusic/flac_files/00ytAjri5yY.flac...
Copying gs://deepclustermusic/flac_files/010KyIQjkTk.flac...
Copying gs://deepclustermusic/flac_files/01Ac0jZ2lr8.flac...
Copying gs://deepclustermusic/flac_files/01JFDoHQwoM.flac...
Copying gs://deepclustermusic/flac_files/02BL9CcY3xg.flac...
Copying gs://deepclustermusic/flac_files/03elWrfAIs0.flac...
Copying gs://deepclustermusic/flac_files/03j7wTQm7KQ.flac...
Copying gs://deepclustermusic/flac_files/04mfKJWDSzI.flac...
Copying gs://deepcluster

In [ ]:
# upload data to google cloud
# !gsutil cp testresults.csv gs://deepclustermusic/

In [ ]:
!pip install fastaudio

In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np

import fastaudio

import imageio
import scipy
from scipy import ndimage
from scipy.signal import convolve2d

import torch
import torch.nn as nn

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:47: UserWarning: "torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. Please remove the use of flag.
  '"torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE" flag is deprecated and will be removed in 0.9.0. '


In [ ]:
# set mel configurations
run_config = dict(
    sample_rate=44100,
    n_fft=4096,
    n_mels=224,
    hop_length=441,
    win_length=1764,
    f_max=20000
)

In [9]:
'''
get frequency features

'''

flacs = list(glob.glob('/content/song_flacs/*.flac'))



In [ ]:
# tensorflow syntax here
def self_attention(in_shape, ch, k=8):
  height, width, channel = in_shape
  x = layers.Input(shape = [height, width, channel])
	
  f = layers.Conv2D(ch // k, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(x)
  f = layers.MaxPooling2D()(f)
  f = layers.Reshape((-1, f.shape[-1]))(f)
	
  g = layers.Conv2D(ch // k, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(x)
  g = layers.Reshape((-1, g.shape[-1]))(g)
	
  h = layers.Conv2D(ch // 2, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(x)
  h = layers.MaxPooling2D()(h)
  h = layers.Reshape((-1, h.shape[-1]))(h)
	
  s = tf.matmul(g, f, transpose_b=True)
  s = keras.layers.Softmax()(s)
		
  o = tf.matmul(s, h)

  o = layers.Reshape((height, width, ch // 2))(o)
  o = layers.Conv2D(channel, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(o)
  o = Scalar()(o)
  o = o + x

  SA = keras.Model(inputs=x, outputs=o)
	
  return SA


class Scalar(layers.Layer):
  def __init__(self):
    super(Scalar, self).__init__()

  def build(self, input_shape):
    self.gamma = tf.Variable(initial_value=tf.zeros(1), trainable=True)
    self._trainable_weights=[self.gamma]

  def call(self, inputs):
    return layers.Rescaling(self.gamma)(inputs)

In [ ]:
def conv_model():
  input = layers.Input(shape=(256, 128))

  image = SpectralNorm(layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same', input_shape=[256, 128, 1]))(input)
  image = layers.LeakyReLU(image)
  image = layers.Dropout(0.3)(image)

  